<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: simón repolt
- Nombre de alumno 2: magdalena de la fuente


### **Link de repositorio de GitHub:** `https://github.com/HexaPulsar/LabMDS`

## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\magda\AppData\Local\Temp\ipykernel_3328\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,FunctionTransformer,MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib
from sklearn_pandas import DataFrameMapper

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score


SEED = 123

In [4]:
X = df.drop("quantity",axis=1)
y = df["quantity"]

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33333, random_state=SEED)

del X_temp
del y_temp
def date2dmy(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df[['day', 'month', 'year']]

date_transformer = FunctionTransformer(date2dmy, validate=False)
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(transformers=[('date', date_transformer, ['date']),
                                               ('num', StandardScaler(), numerical_features),
                                               ('cat',OneHotEncoder(handle_unknown='ignore',sparse_output=False), categorical_features)])
preprocessor.set_output(transform='pandas')

pipe_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', DummyRegressor())])
display(pipe_dummy)

c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_function_transformer.py:345: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('date',
                                                  FunctionTransformer(func=<function date2dmy at 0x000002577E71CD30>),
                                                  ['date']),
                                                 ('num', StandardScaler(),
                                                  Index(['id', 'lat', 'long', 'pop', 'price'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['city', 'shop', 'brand', 'container', 'capacity'], dtype='object'))])),
                ('regressor', DummyRegressor())])

In [6]:
pipe_dummy.fit(X_train, y_train)

y_val_pred = pipe_dummy.predict(X_val)

mae = mean_absolute_error(y_val, y_val_pred)
print(f'MAE del conjunto val para dummy: {mae}')

MAE del conjunto val para dummy: 13254.740836794412


In [7]:
pipe_xgboost = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', XGBRegressor(enable_categorical = True,random_state=SEED))])


pipe_xgboost.fit(X_train, y_train)

y_val_pred_xgboost = pipe_xgboost.predict(X_val)

mae_xgboost = mean_absolute_error(y_val, y_val_pred_xgboost)
print(f'MAE del conjunto val para XGBoost: {mae_xgboost}')


MAE del conjunto val para XGBoost: 2571.7737574535754


In [8]:
joblib.dump     (pipe_dummy, 'pipe_dummy.pkl')
joblib.dump(pipe_dummy.named_steps['regressor'], 'model_dummy.pkl')
joblib.dump(pipe_xgboost, 'pipe_xgboost.pkl')
joblib.dump(pipe_xgboost.named_steps['regressor'], 'model_xgboost.pkl')

['model_xgboost.pkl']

Es posible observar que para los regresores implementados, el MAE varía. Para el Regresor dummy, el valor MAE obtenido en el conjunto de validación es de 13254.74, mientras que para el siguiente regresor XGBoost sin relaciones, el valor desciende a 2571.77. Considerando que un valor menor de MAE es mejor, el regresor que entrega mejores resultados es XGBoost Regressor.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [9]:
pipe_xgboost_const = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('regressor', XGBRegressor(random_state=SEED, 
                                                                monotone_constraints={"num__price": -1},
                                                                enable_categorical = True,))])
X_transformed = preprocessor.transform(X_train)
display(X_transformed)
 
pipe_xgboost_const.fit(X_train, y_train)
y_val_pred_xgboost_const = pipe_xgboost_const.predict(X_val)

mae_xgboost_const = mean_absolute_error(y_val, y_val_pred_xgboost_const)
print(f'MAE del conjunto val para XGBoost con relaciones: {mae_xgboost_const}')

,date__day,date__month,date__year,num__id,num__lat,num__long,num__pop,num__price,cat__city_Athens,cat__city_Irakleion,...,cat__brand_gazoza,cat__brand_kinder-cola,cat__brand_lemon-boost,cat__brand_orange-power,cat__container_can,cat__container_glass,cat__container_plastic,cat__capacity_1.5lt,cat__capacity_330ml,cat__capacity_500ml
3785,31,7,2015,0.020555,0.810835,-0.781462,-0.915526,-0.435075,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
6199,30,11,2017,1.147845,-0.196706,0.414637,1.338319,0.369038,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5432,28,2,2017,0.789203,-0.035621,-1.410704,-0.800417,-0.057386,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2299,28,2,2014,-0.671963,-0.207040,0.387798,1.348347,0.734544,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4568,30,4,2016,0.383324,1.422810,-0.308728,-0.012636,0.113184,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5218,30,11,2016,0.689682,1.422810,-0.308728,-0.012636,0.722361,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4060,31,10,2015,0.148510,0.810835,-0.781462,-0.915526,-0.800582,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1346,30,4,2013,-1.116367,-1.808575,1.729167,-0.942616,-0.569094,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3454,31,3,2015,-0.134459,-0.207040,0.387798,1.344193,2.281854,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


MAE del conjunto val para XGBoost con relaciones: 2721.9707533940623


Para el regresor XGBoost con relaciones, el MAE asciende a 2721.97.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [10]:
import optuna
# Define the objective function for Optuna
# Define the objective function for Optuna
def objective(trial):
    # Split the data into training and validation sets
     

    # Set the hyperparameters to be optimized
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    # Set the XGBoost parameters
    pipe_xgboost_const.named_steps['regressor'].set_params(**xgb_params)
    
     # Set the min_frequency parameter in OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 0.1)
    pipe_xgboost_const.named_steps['preprocessor'].named_transformers_['cat'].set_params(min_frequency=min_frequency)
    
    # Fit the model
    pipe_xgboost_const.fit(X_train, y_train)

    # Make predictions on the validation set
    y_val_pred_xgboost_optuna = pipe_xgboost_const.predict(X_val)

    # Calculate the mean absolute error
    mae_xgboost_optuna = mean_absolute_error(y_val, y_val_pred_xgboost_optuna)

    return mae_xgboost_optuna 

# Create a study and optimize the hyperparameters with TPESampler
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='minimize')
study.optimize(objective, timeout=5*60)  # You can adjust the number of trials


c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-16 14:14:25,143] A new study created in memory with name: no-name-adb130b0-9341-4bfb-8444-1d5718049395
[I 2023-11-16 14:14:30,102] Trial 0 finished with value: 2433.093954023621 and parameters: {'learning_rate': 0.08367859464346475, 'n_estimators': 442, 'max_depth': 7, 'max_leaves': 30, 'min_child_weight': 5, 'reg_alpha': 0.3924644358588971, 'reg_lambda': 0.09445375180367899, 'min_frequency': 0.09014647936242898}. Best is trial 0 with value: 2433.093954023621.
[I 2023-11-16 14:14:37,107] Trial 1 finished with value: 3216.774111465669 and parameters: {'learning_rate': 0.012191967001867855, 'n_estimators': 707, 'max_depth': 9, 'max_leaves': 19, 'min_child_weight': 2, 'reg_alpha': 0.21059588139190877, 're

---
1. `learning_rate`:  valores flotantes en el rango (0.001, 0.1)

> La tasa de aprendizaje controla el tamaño del paso durante el proceso de optimización. Una tasa de aprendizaje más pequeña permite una optimización más precisa pero puede requerir más iteraciones. El rango 0.001, 0.1 es un rango aceptable, pues justamente barre sobre los pasos muy pequeños, y  los pasos considerados grandes.
---

2. `n_estimators`:  valores enteros en el rango (50, 1000)

> Determina el número de rondas de impulso (árboles) en el modelo. Un número más alto de estimadores puede mejorar el rendimiento del modelo, pero hay un equilibrio entre la complejidad del modelo y el tiempo de entrenamiento. El rango (50, 1000) cubre un amplio espectro para encontrar un equilibrio apropiado.

---
3. `max_depth`:  valores enteros en el rango (3, 10)
> Es la profundidad máxima de cada árbol. Un max_depth más alto permite al modelo capturar patrones más complejos en los datos, pero puede causar sobreajuste. El rango (3, 10)es un rango apropiado para encontrar una profundidad que evite el sobreajuste mientras captura los patrones.
----
4. `max_leaves`:  valores enteros en el rango (0, 100)
> controla el número máximo de hojas por árbol. Un valor más alto permite árboles más complejos, sin embargo demasiadas hojas pueden provocar sobreajuste. El rango (0, 100) cubre un amplio espectro de valores.

---
5. `min_child_weight`:  valores enteros en el rango (1, 5)
> Es la suma mínima de peso de instancia necesaria en un hijo. Controla el sobreajuste. El rango (1, 5) es apropiado para encontrar un valor que evite que el modelo sea demasiado sensible a instancias individuales.
---
6. `reg_alpha`:  valores flotantes en el rango (0, 1)
> Término de regularización L1 en los pesos. Ayuda a prevenir el sobreajuste penalizando los pesos grandes. El rango (0, 1) permite ajustar la fuerza de regularización.
---
7. `reg_lambda`:  valores flotantes en el rango (0, 1)
> regularización L2 en los pesos. Al igual que reg_alpha, previene el sobreajuste penalizando los pesos grandes. El rango (0, 1) permite ajustar la fuerza de la regularización.

Los mejores resultados para el estudio son los siguientes:

In [11]:
print(f'N  trials: {len(study.trials)}')
print(f'Mejor trial:')
trial = study.best_trial
print(f'  Valor: {trial.value}') 


N  trials: 35
Mejor trial:
  Valor: 2176.0617377769217


In [12]:
best_params = study.best_params 
for param in best_params:
    print(f'{param}:{best_params[param]}')

learning_rate:0.04261279311889504
n_estimators:995
max_depth:8
max_leaves:88
min_child_weight:5
reg_alpha:0.9755762944748185
reg_lambda:0.5905797992268644
min_frequency:0.05356512540253968


Reasignamos los parámetros del modelo con los recién encontrados

In [13]:
 
pipe_xgboost_const.named_steps['regressor'].set_params(**best_params)
preprocessor.named_transformers_['cat'].set_params(min_frequency = best_params['min_frequency'])

pipe_xgboost_const.fit(X_train, y_train)
y_val_pred_xgboost_const = pipe_xgboost_const.predict(X_val)

mae_xgboost_const = mean_absolute_error(y_val, y_val_pred_xgboost_const)
print(f'MAE del conjunto val para XGBoost con relaciones: {mae_xgboost_const}')

c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [14:19:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_frequency" } are not used.

  warnings.warn(smsg, UserWarning)


MAE del conjunto val para XGBoost con relaciones: 2176.0617377769217


Los resultados de este modelo son mejores que los modelos anteriores. Esto tiene sentido, pues se han elegido parámetros más óptimos que los por defecto seleccionados en las partes anteriores.

In [14]:
joblib.dump(pipe_xgboost_const, 'pipe_xgboost_mejor.pkl')
joblib.dump(pipe_xgboost_const.named_steps['regressor'], 'model_xgboost_const_mejor.pkl')

['model_xgboost_const_mejor.pkl']

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [36]:
import xgboost as xgb

def objective(trial):
     

    # Create the pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor)
    ])

    # Assuming you have a DataFrame X
    X_transformed = pipeline.fit_transform(X_train)
    X_val_trans = pipeline.transform(X_val)
 
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    # Set the XGBoost parameters
    
    reg.set_params(**xgb_params)
    # Set the min_frequency parameter in OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 0.1)
    pipeline.named_steps['preprocessor'].named_transformers_['cat'].set_params(min_frequency=min_frequency)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_1-mae')

    reg.fit(X_transformed, y_train, 
            eval_metric=['mae'], 
            eval_set=[(X_transformed, y_train), (X_val_trans, y_val)], 
            callbacks=[pruning_callback])
    y_val_pred_xgboost_optuna = reg.predict(X_val_trans)
    # Calculate the mean absolute error
    mae_xgboost_optuna = mean_absolute_error(y_val, y_val_pred_xgboost_optuna)

    return mae_xgboost_optuna 

reg = XGBRegressor(random_state=SEED,enable_categorical=True)

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize"
    )
study.optimize(objective, timeout = 5*60)


[0]	validation_0-mae:13061.50414	validation_1-mae:12901.79228
[1]	validation_0-mae:12707.52386	validation_1-mae:12570.55825
[2]	validation_0-mae:12361.21580	validation_1-mae:12245.20768
[3]	validation_0-mae:12036.40210	validation_1-mae:11948.12965
[4]	validation_0-mae:11723.41422	validation_1-mae:11657.06888
[5]	validation_0-mae:11420.32056	validation_1-mae:11378.18818
[6]	validation_0-mae:11133.56921	validation_1-mae:11113.15234
[7]	validation_0-mae:10853.44470	validation_1-mae:10858.55093
[8]	validation_0-mae:10589.26103	validation_1-mae:10612.53921
[9]	validation_0-mae:10331.93145	validation_1-mae:10381.12719
[10]	validation_0-mae:10088.12914	validation_1-mae:10157.62198
[11]	validation_0-mae:9855.29209	validation_1-mae:9936.02127
[12]	validation_0-mae:9622.84416	validation_1-mae:9730.11529
[13]	validation_0-mae:9407.21392	validation_1-mae:9524.92779
[14]	validation_0-mae:9191.67449	validation_1-mae:9330.95245


[15]	validation_0-mae:8985.13390	validation_1-mae:9150.22859
[16]	validation_0-mae:8793.11592	validation_1-mae:8970.62612
[17]	validation_0-mae:8596.70580	validation_1-mae:8783.29026
[18]	validation_0-mae:8404.92901	validation_1-mae:8613.50932
[19]	validation_0-mae:8219.62953	validation_1-mae:8444.76231
[20]	validation_0-mae:8050.37184	validation_1-mae:8287.82386
[21]	validation_0-mae:7888.82661	validation_1-mae:8133.46598
[22]	validation_0-mae:7722.53352	validation_1-mae:7981.44351
[23]	validation_0-mae:7560.69008	validation_1-mae:7837.07957
[24]	validation_0-mae:7412.73673	validation_1-mae:7701.74390
[25]	validation_0-mae:7270.28720	validation_1-mae:7572.26728
[26]	validation_0-mae:7126.08691	validation_1-mae:7444.54321
[27]	validation_0-mae:6982.55099	validation_1-mae:7312.63106
[28]	validation_0-mae:6849.92763	validation_1-mae:7191.19514
[29]	validation_0-mae:6711.78223	validation_1-mae:7067.65469
[30]	validation_0-mae:6579.51980	validation_1-mae:6938.61011
[31]	validation_0-mae:64

c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[34]	validation_0-mae:6113.31341	validation_1-mae:6517.47292
[35]	validation_0-mae:5995.07731	validation_1-mae:6411.31957
[36]	validation_0-mae:5894.50350	validation_1-mae:6318.25692
[37]	validation_0-mae:5789.31776	validation_1-mae:6221.94660
[38]	validation_0-mae:5692.56160	validation_1-mae:6134.76835
[39]	validation_0-mae:5594.37901	validation_1-mae:6041.35916
[40]	validation_0-mae:5504.92497	validation_1-mae:5960.11481
[41]	validation_0-mae:5414.80648	validation_1-mae:5872.94896
[42]	validation_0-mae:5321.53330	validation_1-mae:5784.06823
[43]	validation_0-mae:5242.25074	validation_1-mae:5713.88641
[44]	validation_0-mae:5153.09496	validation_1-mae:5635.64967
[45]	validation_0-mae:5071.56960	validation_1-mae:5561.25894
[46]	validation_0-mae:4989.95622	validation_1-mae:5486.87727
[47]	validation_0-mae:4910.74323	validation_1-mae:5412.43455
[48]	validation_0-mae:4833.72032	validation_1-mae:5341.58742
[49]	validation_0-mae:4767.55117	validation_1-mae:5280.45857
[50]	validation_0-mae:46

c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


...

[0]	validation_0-mae:12543.70099	validation_1-mae:12422.74647
[1]	validation_0-mae:11733.22758	validation_1-mae:11648.54396
[2]	validation_0-mae:10983.62016	validation_1-mae:10962.38583
[3]	validation_0-mae:10316.19164	validation_1-mae:10346.11992
[4]	validation_0-mae:9700.12179	validation_1-mae:9785.51494
[5]	validation_0-mae:9131.87834	validation_1-mae:9270.55528
[6]	validation_0-mae:8614.27251	validation_1-mae:8798.35081


c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit

[0]	validation_0-mae:12459.11200	validation_1-mae:12337.30318
[1]	validation_0-mae:11566.67759	validation_1-mae:11483.91790
[2]	validation_0-mae:10776.73673	validation_1-mae:10754.24411
[3]	validation_0-mae:10051.31296	validation_1-mae:10090.21164
[4]	validation_0-mae:9391.34232	validation_1-mae:9484.48399
[5]	validation_0-mae:8788.22090	validation_1-mae:8941.34035
[6]	validation_0-mae:8245.61475	validation_1-mae:8436.23968
[7]	validation_0-mae:7735.02374	validation_1-mae:7992.58015
[8]	validation_0-mae:7282.92981	validation_1-mae:7573.01568
[9]	validation_0-mae:6854.05665	validation_1-mae:7197.01623
[10]	validation_0-mae:6473.42747	validation_1-mae:6862.33726
[11]	validation_0-mae:6120.45573	validation_1-mae:6532.43801
[12]	validation_0-mae:5786.73920	validation_1-mae:6224.81149
[13]	validation_0-mae:5470.77524	validation_1-mae:5938.44803
[14]	validation_0-mae:5204.62757	validation_1-mae:5701.10945
[15]	validation_0-mae:4940.63566	validation_1-mae:5448.72896
[16]	validation_0-mae:4705

c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[36]	validation_0-mae:2402.67976	validation_1-mae:3194.40397
[37]	validation_0-mae:2357.76059	validation_1-mae:3163.05946
[38]	validation_0-mae:2312.86207	validation_1-mae:3119.48277
[39]	validation_0-mae:2271.82293	validation_1-mae:3088.61437
[40]	validation_0-mae:2202.67248	validation_1-mae:3022.79419
[41]	validation_0-mae:2169.51214	validation_1-mae:2997.00743
[42]	validation_0-mae:2120.55745	validation_1-mae:2963.36546
[43]	validation_0-mae:2087.98988	validation_1-mae:2939.30776
[44]	validation_0-mae:2062.57686	validation_1-mae:2922.46262
[45]	validation_0-mae:2033.78492	validation_1-mae:2899.31474
[46]	validation_0-mae:1998.88332	validation_1-mae:2872.93686
[47]	validation_0-mae:1975.77077	validation_1-mae:2856.53272
[48]	validation_0-mae:1942.53266	validation_1-mae:2830.04952
[49]	validation_0-mae:1914.99106	validation_1-mae:2813.22536
[50]	validation_0-mae:1893.62976	validation_1-mae:2797.28888
[51]	validation_0-mae:1872.77758	validation_1-mae:2782.51843
[52]	validation_0-mae:18

In [38]:
#print(study.best_trial)
best_params = study.best_params 
reg.set_params(**best_params)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.08205838634407842,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=82,
             min_child_weight=4, min_frequency=0.02843798141447011, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=770,
             n_jobs=None, num_parallel_tree=None, ...)

In [41]:

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# Assuming you have a DataFrame X
X_transformed = pipeline.fit_transform(X_train)
X_val_trans = pipeline.transform(X_val)

y_val_pred_xgboost_const = reg.predict(X_val_trans)

mae_xgboost_const = mean_absolute_error(y_val, y_val_pred_xgboost_const)
print(f'MAE del conjunto val para XGBoost con relaciones: {mae_xgboost_const}')

MAE del conjunto val para XGBoost con relaciones: 2081.299073903054


c:\Users\magda\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [15:41:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_frequency" } are not used.

  warnings.warn(smsg, UserWarning)


## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
# Inserte su código acá

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>